In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [4]:
# lo primero que hacemos es cargar los dataframes que vamos a usar 

df_clientes = pd.read_csv("../data/clientes.csv", index_col = 0)
df_clientes.head()

,first_name,last_name,email,gender,City,Country,Address
id,,,,,,,
1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle


In [5]:
df_ventas = pd.read_csv("../data/ventas.csv", index_col = 0)
df_ventas.head()

,ID_Producto,Fecha_Venta,Cantidad,Total
ID_Cliente,,,,
723,A1,2023-11-22,2,17.98
498,C3,2023-11-21,1,5.49
121,D4,2023-11-20,3,32.97
885,L12,2023-11-19,1,6.49
347,Q17,2023-11-18,2,7.98


In [12]:
df_productos = pd.read_csv("../data/productos.csv", sep=None, index_col = 0)
df_productos.head()

,Nombre_Producto,Categoría,Precio,Origen,Descripción
ID,,,,,
A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate,mozzarella f..."
B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos,una de..."
C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...
E5,Orecchiette,Productos Secos,4.29,Italia,Pequeñas pastas con forma de orecchiette ideal...


In [13]:
df_clientes.shape

(1000, 7)

In [14]:
df_ventas.shape

(100, 4)

In [15]:
df_productos.shape

(33, 5)

In [16]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  1000 non-null   object
 1   last_name   1000 non-null   object
 2   email       973 non-null    object
 3   gender      923 non-null    object
 4   City        876 non-null    object
 5   Country     846 non-null    object
 6   Address     959 non-null    object
dtypes: object(7)
memory usage: 62.5+ KB


In [19]:
df_clientes['City'].isnull().sum()

124

In [17]:
df_clientes['City'].unique()

array(['Palma De Mallorca', 'Lugo', nan, 'Leon',
       "Hospitalet De Llobregat, L'", 'Lleida', 'Santander', 'Malaga',
       'Valladolid', 'Murcia', 'Pamplona/Iruña', 'Barcelona', 'Palencia',
       'Pontevedra', 'Girona', 'Fuenlabrada', 'Jaen', 'Almeria',
       'Elx/Elche', 'Getafe', 'Sevilla', 'Coruña, A', 'Zaragoza',
       'Palmas De Gran Canaria, Las', 'Cadiz', 'Tarragona',
       'Santiago De Compostela', 'Donostia-San Sebastian', 'Cartagena',
       'Madrid', 'Ceuta', 'Vigo', 'Valencia', 'Sabadell', 'Granada',
       'Albacete', 'Teruel', 'Aviles',
       'Castellon De La Plana/Castello De La Pla', 'Vitoria-Gasteiz',
       'Zamora', 'Santa Cruz De Tenerife', 'Logroño', 'Torrevieja',
       'Leganes', 'Gijon', 'Toledo', 'Badajoz', 'Dos Hermanas', 'Ourense',
       'Ferrol', 'Salamanca', 'Sant Cugat Del Valles', 'Huesca',
       'Alicante/Alacant', 'Telde'], dtype=object)

In [18]:
df_clientes['City'].value_counts(dropna=False, normalize=True)*100

City
NaN                                         12.4
Madrid                                       5.1
Granada                                      4.2
Palma De Mallorca                            4.1
Malaga                                       3.6
Vitoria-Gasteiz                              3.2
Palmas De Gran Canaria, Las                  3.1
Valencia                                     2.7
Pamplona/Iruña                               2.7
Pontevedra                                   2.4
Cadiz                                        2.4
Sevilla                                      2.4
Lleida                                       2.3
Zamora                                       2.2
Valladolid                                   2.2
Santander                                    2.2
Donostia-San Sebastian                       2.1
Logroño                                      2.0
Albacete                                     2.0
Leon                                         2.0
Vigo           

In [34]:
df_clientes_loc = df_clientes.loc[pd.isna(df_clientes['City']), :]

In [35]:
df_clientes_loc

,first_name,last_name,email,gender,City,Country,Address
id,,,,,,,
3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
9,Tymon,Darragh,tdarragh8@free.fr,Male,NaN,Spain,41 Trailsway Way
11,Chase,Godilington,cgodilingtona@spiegel.de,Male,NaN,NaN,13 Bluestem Court
19,Sena,MacElane,smacelanei@youtu.be,Female,NaN,Spain,8032 Claremont Trail
33,Adoree,Cutchey,acutcheyw@seesaa.net,NaN,NaN,Spain,917 Marcy Parkway
...,...,...,...,...,...,...,...
943,Maritsa,Doale,mdoaleq6@google.nl,NaN,NaN,Spain,7 Toban Center
962,Domenic,Adger,dadgerqp@time.com,Male,NaN,Spain,327 Gateway Crossing
965,Marijn,St Clair,mstclairqs@technorati.com,Male,NaN,Spain,57 Jay Drive


In [37]:
df_address_loc = df_clientes.loc[pd.isna(df_clientes['Address']), :]

In [38]:
df_address_loc

,first_name,last_name,email,gender,City,Country,Address
id,,,,,,,
10,Igor,Dwelley,idwelley9@flickr.com,Male,Lugo,Spain,NaN
35,Egon,Lafflin,elaffliny@a8.net,Male,Sevilla,Spain,NaN
73,Reinald,Platts,rplatts20@phpbb.com,NaN,Malaga,Spain,NaN
74,Tate,Capehorn,tcapehorn21@comsenz.com,Male,Barcelona,Spain,NaN
82,Uta,Crasford,NaN,Female,Lugo,Spain,NaN
115,Carole,Biasetti,cbiasetti36@soup.io,Female,NaN,Spain,NaN
148,Carling,Pharo,cpharo43@youku.com,Male,Donostia-San Sebastian,Spain,NaN
157,Nancie,Radcliffe,nradcliffe4c@mozilla.org,Female,NaN,Spain,NaN
168,Storm,Musterd,NaN,Female,Badajoz,Spain,NaN


In [39]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, A1 to GG33
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Nombre_Producto  33 non-null     object 
 1   Categoría        33 non-null     object 
 2   Precio           33 non-null     float64
 3   Origen           33 non-null     object 
 4   Descripción      33 non-null     object 
dtypes: float64(1), object(4)
memory usage: 1.5+ KB


In [40]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 723 to 987
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_Producto  100 non-null    object 
 1   Fecha_Venta  100 non-null    object 
 2   Cantidad     100 non-null    int64  
 3   Total        100 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.9+ KB


In [41]:
df_ventas['Fecha_Venta'].unique()

array(['2023-11-22', '2023-11-21', '2023-11-20', '2023-11-19',
       '2023-11-18', '2023-11-17', '2023-11-16', '2023-11-15',
       '2023-11-14', '2023-11-13', '2023-11-08', '2023-11-07',
       '2023-11-06', '2023-11-05', '2023-11-04', '2023-11-03',
       '2023-11-02', '2023-11-01', '2023-10-31', '2023-10-30',
       '2023-10-29', '2023-10-28', '2023-10-27', '2023-10-26',
       '2023-10-25', '2023-10-24', '2023-10-23', '2023-10-22',
       '2023-10-21', '2023-10-20', '2023-10-19', '2023-10-18',
       '2023-10-17', '2023-10-16', '2023-10-15', '2023-10-14',
       '2023-10-13', '2023-10-12', '2023-10-11', '2023-10-10',
       '2023-10-09', '2023-10-08', '2023-10-07', '2023-10-06',
       '2023-10-05', '2023-10-04', '2023-10-03', '2023-10-02',
       '2023-10-01', '2023-09-30', '2023-09-29', '2023-09-28',
       '2023-09-27', '2023-09-26', '2023-09-25', '2023-09-24',
       '2023-09-23', '2023-09-22', '2023-09-21', '2023-09-20',
       '2023-09-19', '2023-09-18', '2023-09-17', '2023-

In [42]:
df_ventas['Fecha_Venta'] = pd.to_datetime(df_ventas['Fecha_Venta'])
# nuevamente usamos dtypes sobre el dataframe para corroborar de que tipo es cada columna.
df_ventas.dtypes

ID_Producto            object
Fecha_Venta    datetime64[ns]
Cantidad                int64
Total                 float64
dtype: object

In [46]:
df_clientes.duplicated().sum()

0

In [47]:
df_ventas.duplicated().sum()

0

In [48]:
df_productos.duplicated().sum()

0

In [49]:
df_clientes['Country'].unique()

array(['Spain', nan], dtype=object)

In [50]:
df_clientes['Country'] = df_clientes['Country'].fillna('Spain')

In [51]:
df_clientes['Country'].unique()

array(['Spain'], dtype=object)